# Задача 

В следующей ячейке написал код по обучению модели машинного обучения для классификации Ирисов.
Необходимо реализовать веб-сервис на Flask и обработчик на Celery таким образом, чтобы получившаяся система позволяла использовать эту модель для классификации через сеть. 

In [1]:
from sklearn.datasets import load_iris
import pickle
from sklearn.linear_model import LogisticRegression


X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)

После того, как вы реализуете свой веб-сервис, достаточно будет его запустить и нажать кнопку "Отправить решение". После нажатия автоматически запустится скрипт `check-server.py`, который создаст файл `result.json`. 

Сам скрипт можно использовать для проверки корректности своего решения.

In [2]:
raw_data = pickle.dumps(clf)

with open('fmeter-model.pickle', 'wb') as f:
    f.write(raw_data)

In [3]:
%%writefile server.py
from celery import Celery
from celery.result import AsyncResult
import time
from flask import Flask, request
import json
import pickle
import re

celery_app = Celery('server', backend='redis://localhost', broker='redis://localhost')  # и брокер и база - redis
app = Flask(__name__)  # Основной объект приложения Flask


def load_model(pickle_path):
    with open(pickle_path, 'rb') as f:
        raw_data = f.read()
        model = pickle.loads(raw_data)
    return model

model = load_model('fmeter-model.pickle')

@celery_app.task
def predict(data_iris):
    result = int(model.predict([data_iris])[0])
    return result

@app.route('/iris', methods=["GET", "POST"])
def predict_handler():
    if request.method == 'POST':
        data = request.get_json(force=True) 
        task = predict.delay(data['iris']) 
        response = {
            "task_id": task.id
        }
        return json.dumps(response)
    
@app.route('/iris/<task_id>')
def predict_check_handler(task_id):
    task = AsyncResult(task_id, app=celery_app)
    if task.ready():
        response = {
            "status": "DONE",
            "result": task.result
        }
    else:
        response = {
            "status": "IN_PROGRESS"
        }
    return json.dumps(response)

if __name__ == '__main__':
    app.run("0.0.0.0", 8000)  # Запускаем сервер на 8000 порту

Overwriting server.py


In [4]:
! start-worker.sh

Success!


In [5]:
! launch-server.sh server.py

Success!


In [6]:
! cat $(which check-server.py)

#!/usr/bin/env python3

import requests
import json
import time

questions = [
    [4.6, 3.1, 1.5, 0.2],
    [5.2, 2.7, 3.9, 1.4],
    [6.9, 3.1, 5.1, 2.3]
]

result = []
for q in questions:
    data = {
        'iris': q
    }

    response = requests.post("http://localhost:8000/iris", json=data)
    task_id = response.json()['task_id']
    status = "IN_PROGRESS"
    while status != "DONE":
        time.sleep(2.0)
        r = requests.get('http://localhost:8000/iris/{}'.format(task_id))
        status = r.json()['status']

    result.append(r.json()['result'])

with open('/home/jovyan/work/result.json', 'w') as f:
    f.write(json.dumps(result, indent=4))

In [8]:
! cat log.txt

 * Serving Flask app "server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2021 06:56:42] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2021 06:56:44] "GET /iris/ad8c3716-4f44-40cf-b347-c69ae56aa021 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2021 06:56:44] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2021 06:56:46] "GET /iris/92056716-cbf3-4759-9d9c-b5cb9ed858a9 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2021 06:56:46] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2021 06:56:48] "GET /iris/09f8f64e-bff6-4ab4-ba40-5b01e734a72d HTTP/1.1" 200 -
